In [1]:
from UTILITY_quickstart import *
from scipy.optimize import minimize

#Start from golden lattice
tao = initializeTao(
    loadDefaultLatticeTF = False
)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Not using setLattice(). Golden lattice
WARNING! No beam file is specified!
Number of macro particles defined by input file


In [2]:
def rampToZero(val, thresh, scale = 1):
    return (max(val, thresh) - thresh) / scale
    
def finalFocusSolverObjectiveSpecial(params, tao, ele, s_offset1, targetBetaX1, targetAlphaX1, targetBetaY1, targetAlphaY1, s_offset2, targetBetaX2, targetAlphaX2, targetBetaY2, targetAlphaY2):

    Q5FFkG1, Q4FFkG1, Q3FFkG1, Q2FFkG1, Q1FFkG1, Q0FFkG1, Q5FFkG2, Q4FFkG2, Q3FFkG2, Q2FFkG2, Q1FFkG2, Q0FFkG2  = params
    quadNameList = ["Q5FF", "Q4FF", "Q3FF", "Q2FF", "Q1FF", "Q0FF"]
    quadErrorSigma = 100e-6 
    allErrors = []
    numMonteCarlo = 10
    
    
    #Loop random magnet error
            #Loop twiss target [1,2]
                #Get twiss at point [1,2]
                #Get centroid at PENT
    
    
    for iteration in range(numMonteCarlo):
        #Prevent recalculation until changes are made
        tao.cmd("set global lattice_calc_on = F")
        
        for quad in quadNameList:
            setXOffset(tao, quad, quadErrorSigma * np.random.normal())
            setYOffset(tao, quad, quadErrorSigma * np.random.normal())
        
        #Reenable lattice calculations
        tao.cmd("set global lattice_calc_on = T")
        
        
        
        
        
        #Prevent recalculation until changes are made
        tao.cmd("set global lattice_calc_on = F")
        
        setQuadkG(tao, "Q5FF", Q5FFkG1)
        setQuadkG(tao, "Q4FF", Q4FFkG1)
        setQuadkG(tao, "Q3FF", Q3FFkG1)
        setQuadkG(tao, "Q2FF", Q2FFkG1)
        setQuadkG(tao, "Q1FF", Q1FFkG1)
        setQuadkG(tao, "Q0FF", Q0FFkG1)
        
        #Reenable lattice calculations
        tao.cmd("set global lattice_calc_on = T")
        
        resultTwiss1  = tao.twiss_at_s(ele = ele, s_offset = s_offset1)
        resultBetaX1  = resultTwiss1["beta_a"]
        resultAlphaX1 = resultTwiss1["alpha_a"]
        resultBetaY1  = resultTwiss1["beta_b"]
        resultAlphaY1 = resultTwiss1["alpha_b"]
        
        PENTCentroidX1 = tao.ele_orbit("PENT")['x']
        PENTCentroidY1 = tao.ele_orbit("PENT")['y']
        
        
        #Prevent recalculation until changes are made
        tao.cmd("set global lattice_calc_on = F")
        
        setQuadkG(tao, "Q5FF", Q5FFkG2)
        setQuadkG(tao, "Q4FF", Q4FFkG2)
        setQuadkG(tao, "Q3FF", Q3FFkG2)
        setQuadkG(tao, "Q2FF", Q2FFkG2)
        setQuadkG(tao, "Q1FF", Q1FFkG2)
        setQuadkG(tao, "Q0FF", Q0FFkG2)
        
        #Reenable lattice calculations
        tao.cmd("set global lattice_calc_on = T")
        
        resultTwiss2  = tao.twiss_at_s(ele = ele, s_offset = s_offset2)
        resultBetaX2  = resultTwiss2["beta_a"]
        resultAlphaX2 = resultTwiss2["alpha_a"]
        resultBetaY2  = resultTwiss2["beta_b"]
        resultAlphaY2 = resultTwiss2["alpha_b"]
        
        PENTCentroidX2 = tao.ele_orbit("PENT")['x']
        PENTCentroidY2 = tao.ele_orbit("PENT")['y']
        
        
        twissError1 = (resultBetaX1 - targetBetaX1) ** 2 + (resultAlphaX1 - targetAlphaX1) ** 2 + (resultBetaY1 - targetBetaY1) ** 2 + (resultAlphaY1 - targetAlphaY1) ** 2
        twissError2 = (resultBetaX2 - targetBetaX2) ** 2 + (resultAlphaX2 - targetAlphaX2) ** 2 + (resultBetaY2 - targetBetaY2) ** 2 + (resultAlphaY2 - targetAlphaY2) ** 2
        centroidShiftError = np.sqrt( ( PENTCentroidX1 - PENTCentroidX2 ) ** 2 + ( PENTCentroidY1 - PENTCentroidY2 ) ** 2 )

        
        totalError = twissError1 + twissError2 + rampToZero(centroidShiftError, 20e-6, scale = 1e-6)
    
        allErrors.append(totalError)

    #print(allErrors)
    
    return np.median(allErrors)




def finalFocusSolverSpecial(
    tao,
    ele = "PENT",
    s_offset1 = -0.5,
    targetBetaX1 = 0.5,
    targetAlphaX1 = 0,
    targetBetaY1 = 0.5,
    targetAlphaY1 = 0,
    s_offset2 = 0.5,
    targetBetaX2 = 0.5,
    targetAlphaX2 = 0,
    targetBetaY2 = 0.5,
    targetAlphaY2 = 0,
    verbose = False
):


    quadNameList = ["Q5FF", "Q4FF", "Q3FF", "Q2FF", "Q1FF", "Q0FF", "Q5FF", "Q4FF", "Q3FF", "Q2FF", "Q1FF", "Q0FF"] 
    initialGuess = [getQuadkG(tao, name) for name in quadNameList]

    #For now, just hardcoding bounds... could generalize if required
    #From "bounds.yml" as of 2025-01-10-11-11-35
    # Q5FFkGBounds: (-256, 0)  #BCON = -70
    # Q4FFkGBounds: (-446, 0)  #BCON = -71
    # Q3FFkGBounds: (0, 457)   #BCON = 106
    # Q2FFkGBounds: (0, 167)   #BCON = 112
    # Q1FFkGBounds: (-257, 0)  #BCON = -225
    # Q0FFkGBounds: (0, 167)   #BCON = 112

    bounds = [(-256,0), (-446,0), (0,457), (0,167), (-257,0), (0,167), (-256,0), (-446,0), (0,457), (0,167), (-257,0), (0,167)]


    # Perform optimization using Nelder-Mead
    result = minimize(
        finalFocusSolverObjectiveSpecial, 
        initialGuess, 
        method='Nelder-Mead',
        options = {"maxiter" : 500},
        bounds = bounds,
        args = (tao, ele, s_offset1, targetBetaX1, targetAlphaX1, targetBetaY1, targetAlphaY1, s_offset2, targetBetaX2, targetAlphaX2, targetBetaY2, targetAlphaY2)
    )



    if verbose:
        print("Optimization Results:")
        print(f"Optimal Parameters: {result.x}")
        print(f"Objective Function Value at Optimal Parameters: {result.fun}")
        print(f"Number of Iterations: {result.nit}")
        print(f"Converged: {result.success}")

    quadVariableNameList = ["Q5FFkG1", "Q4FFkG1", "Q3FFkG1", "Q2FFkG1", "Q1FFkG1", "Q0FFkG1", "Q5FFkG2", "Q4FFkG2", "Q3FFkG2", "Q2FFkG2", "Q1FFkG2", "Q0FFkG2"] 

    
    
    return { quadVariableNameList[i] : result.x[i] for i in range(len(quadVariableNameList)) }






sol = finalFocusSolverSpecial(tao, verbose = True)

Optimization Results:
Optimal Parameters: [ -72.73496506  -78.74485096  103.41489669  126.14086929 -235.79533777
  126.43676866  -72.29551366  -83.18686511   95.72341313  125.32662412
 -233.80468069  127.54862597]
Objective Function Value at Optimal Parameters: 1.5680799571614088
Number of Iterations: 500
Converged: False


In [3]:
Q5FFkG1, Q4FFkG1, Q3FFkG1, Q2FFkG1, Q1FFkG1, Q0FFkG1, Q5FFkG2, Q4FFkG2, Q3FFkG2, Q2FFkG2, Q1FFkG2, Q0FFkG2  = list(sol.values())

In [4]:
#Prevent recalculation until changes are made
tao.cmd("set global lattice_calc_on = F")

setQuadkG(tao, "Q5FF", Q5FFkG1)
setQuadkG(tao, "Q4FF", Q4FFkG1)
setQuadkG(tao, "Q3FF", Q3FFkG1)
setQuadkG(tao, "Q2FF", Q2FFkG1)
setQuadkG(tao, "Q1FF", Q1FFkG1)
setQuadkG(tao, "Q0FF", Q0FFkG1)

#Reenable lattice calculations
tao.cmd("set global lattice_calc_on = T")


PENTCentroidX1 = tao.ele_orbit("PENT")['x']
PENTCentroidY1 = tao.ele_orbit("PENT")['y']


tao.twiss_at_s(ele = "PENT", s_offset = -0.5)

{'beta_a': 0.575549967856467,
 'alpha_a': 0.114290906061348,
 'gamma_a': 1.76016413480361,
 'phi_a': 72.9900632944764,
 'eta_a': 0.000323570370916389,
 'etap_a': -0.000107871614953764,
 'beta_b': 0.875233402964798,
 'alpha_b': 0.815565173136434,
 'gamma_b': 1.90251714113342,
 'phi_b': 59.6782686391464,
 'eta_b': 0.00198205958167567,
 'etap_b': 0.000625547407575545,
 'c_mat11': -3.10853044579465e-09,
 'c_mat12': 3.35852565686131e-08,
 'c_mat21': 7.78989861868682e-09,
 'c_mat22': -4.12654017826897e-08,
 'gamma_c': 1.0}

In [5]:
#Prevent recalculation until changes are made
tao.cmd("set global lattice_calc_on = F")

setQuadkG(tao, "Q5FF", Q5FFkG2)
setQuadkG(tao, "Q4FF", Q4FFkG2)
setQuadkG(tao, "Q3FF", Q3FFkG2)
setQuadkG(tao, "Q2FF", Q2FFkG2)
setQuadkG(tao, "Q1FF", Q1FFkG2)
setQuadkG(tao, "Q0FF", Q0FFkG2)

#Reenable lattice calculations
tao.cmd("set global lattice_calc_on = T")


PENTCentroidX2 = tao.ele_orbit("PENT")['x']
PENTCentroidY2 = tao.ele_orbit("PENT")['y']


tao.twiss_at_s(ele = "PENT", s_offset = 0.5)

{'beta_a': 0.463850052231495,
 'alpha_a': -0.241974530450718,
 'gamma_a': 2.28209885564172,
 'phi_a': 73.3350877674684,
 'eta_a': 0.000286730207637898,
 'etap_a': -0.000108149274676035,
 'beta_b': 0.762319544928577,
 'alpha_b': -0.784023805528522,
 'gamma_b': 2.11813187576964,
 'phi_b': 61.1256498123634,
 'eta_b': 0.00235934942993691,
 'etap_b': 0.000539035927435367,
 'c_mat11': -2.45576731733336e-09,
 'c_mat12': 2.62341144134345e-08,
 'c_mat21': 6.92930656400528e-09,
 'c_mat22': -3.65572338605756e-08,
 'gamma_c': 1.0}

In [6]:
print(1e6 * PENTCentroidX1)
print(1e6 * PENTCentroidY1)
print(1e6 * PENTCentroidX2)
print(1e6 * PENTCentroidY2)
1e6 * np.sqrt( ( PENTCentroidX1 - PENTCentroidX2 ) ** 2 + ( PENTCentroidY1 - PENTCentroidY2 ) ** 2 )

-584.2617659866099
-4.602857049640931
-575.520529892941
33.736257981304604


39.32298246320254